In [ ]:
import epxexec
from epxexec import epxresults
from epxexec import fred_job
from epxexec.visual.utils import default_plotly_template

from pathlib import Path
import os

import pandas as pd
import plotly.express as px
import plotly.io as pio

pio.templates["epistemix"] = default_plotly_template()
pio.templates.default = "epistemix"

os.environ["FRED_PROJECT"] = str(Path.cwd())
path_to_fred_project = os.environ["FRED_PROJECT"]

In [ ]:
mpox_job = fred_job(program="model/main_monkeypox.fred")

In [ ]:
mpox_job.get_job_state_table(condition="BATHHOUSE",state="Visit", count_type="new")

In [ ]:
susceptible = mpox_job.get_job_state_table(
    condition="MPOX",
    state="Susceptible",
    count_type="new"
)

exposed = mpox_job.get_job_state_table(
    condition="MPOX",
    state="Exposed",
    count_type="new"
)

infectious = mpox_job.get_job_state_table(
    condition="MPOX",
    state="Infectious",
    count_type="new"
)


recovered = mpox_job.get_job_state_table(
    condition="MPOX",
    state="Recovered",
    count_type="new"
)

In [ ]:
susceptible

In [ ]:
susceptible.to_csv("sus_counts.csv")

In [ ]:
# rename the 'new' counts column to the name of the state
susceptible.rename(columns={'new':'Susceptible'},inplace=True)
exposed.rename(columns={'new':'Exposed'},inplace=True)
infectious.rename(columns={'new':'Infectious'},inplace=True)
recovered.rename(columns={'new':'Recovered'},inplace=True)

# build up the merged dataframe with counts for all states
all_states = pd.merge(susceptible,exposed)
all_states = pd.merge(all_states,infectious)
all_states = pd.merge(all_states,recovered)

#retrieve the sim_date from the column and add to the all_states frame.
all_states['sim_date']=mpox_job.get_job_date_table().sim_date

In [ ]:
all_states.to_csv("all_mpox_data_vals1.csv")

In [ ]:
all_states['Exposed_7day_avg'] = all_states.Exposed.rolling(7).mean()
all_states['Recovered_7day_avg'] = all_states.Recovered.rolling(7).mean()
all_states['Infectious_7day_avg'] = all_states.Infectious.rolling(7).mean()

all_state_7day_df = all_states[~all_states['Exposed_7day_avg'].isnull()]

fig = px.line(
    all_state_7day_df,
    x='sim_date',
    y=['Exposed_7day_avg','Infectious_7day_avg','Recovered_7day_avg'], # pass a list to plot all states],
    title="Mpox outbreak in County")
fig.update_layout( # customize font and legend orientation & position
    font_family="Epistemix Label",
    yaxis_title="New infections per day, 7 day average",
    xaxis_title="Date",
    legend_title="State",
    title_font_size=30
)
fig.show()